In [ ]:
# Installs

pip install gitpython beautifulsoup4 requests

In [ ]:
# Imports

import requests, time
from bs4 import BeautifulSoup
import os, json
from collections import defaultdict

In [ ]:
# Scrape topic from Github

topic_set = set()

for i in range(7):
    time.sleep(1)
    page = requests.get(f"https://github.com/topics?page={i}")
    # print(soup)
    soup = BeautifulSoup(page.content, 'html.parser')

    for find in soup.find_all('a'):
        topic = find['href']
        if topic.startswith("/topics/"):
            topic_set.add(topic)

    print(i, len(topic_set))

print("Have topic set")


In [ ]:
# Scrape repo names from topics - doesn't work
topic_set = set()
topic_set.add("/topics/cpp")

for topic in topic_set:
    print(topic)
    time.sleep(1)
    page = requests.get(f"https://github.com{topic}?o=desc&s=stars")
    soup = BeautifulSoup(page.content, 'html.parser')

    for a in soup.find_all('a', {"data-ga-click":"Explore, go to repository, location:explore feed"}):
        print(a['href'])
    break

In [ ]:
# Get repos

repos = [line.strip() for line in open('npm-repos.txt')]

repo_contrib_dict = defaultdict(list)

for repo in repos:
    repo_name = '{}/{}'.format(*repo.split("/")[-2:])

    stream = os.popen(f'''
        rm -rf git-test > /dev/null;
        git clone {repo} git-test > /dev/null;
        cd git-test;
        git log --all --format=\'%aE\' | sort -u
    ''')
    contributor_emails = stream.read().split("\n")[:-1]

    for email in contributor_emails:
        repo_contrib_dict[email].append(repo_name)

    print(f"Finished {repo}")

# Remove all users
for key in [key for key in repo_contrib_dict if len(repo_contrib_dict[key]) == 1]: del repo_contrib_dict[key] 

with open('contributors-repos.json', 'w') as outfile:
    json.dump(repo_contrib_dict, outfile)

print("All done")

In [ ]:
def explore_node_page(package):
    page = requests.get(f"https://www.npmjs.com{package}")
    soup = BeautifulSoup(page.content, 'html.parser')

    repository = soup.find('h3', string="Repository")
    git_link = repository.parent.findNext('a')['href']
    return git_link

In [ ]:
# Get node repo list
url = "https://www.npmjs.com/browse/depended"
counter = len([line.strip() for line in open('npm-repos.txt')])
print(f"starting at {counter}")
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

while (counter < 1000):
    page = requests.get(f"https://www.npmjs.com/browse/depended?offset={counter}", headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    for a in soup.find_all('a'):
        if a['href'].startswith("/package/"):
            time.sleep(0.5)
            package = a['href']
            print(f"Working on {package}")
            counter += 1
            git_repo = explore_node_page(package)

            if git_repo not in [line.strip() for line in open('npm-repos.txt')]:
                with open('npm-repos.txt', 'a') as outfile:
                    outfile.write(git_repo + '\n')
            else:
                print(f"duplicate url: {git_repo}")

    git_repo_list = []